# Return signals regression

In [1]:
import pandas as pd
import numpy as np
import sys, os

In [3]:
dataset = pd.read_pickle("data/all_tickers_features.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'data/all_tickers_features.pkl'

## Pipeline de preprocesamiento

In [ ]:
dataset.columns

In [ ]:
continuous_features = ['Open', 'High', 'Low', 'Close', 'Volume',
                       'MACD', 'RSI', 'BB_High', 'BB_Mid', 'BB_Low',
                       'ATR', 'NATR', 'Currency_Volume', 'Adj Close']
categorical_features = ['Month', 'Weekday', 'Ticker', 'Currency']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_union

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class NoTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X

In [ ]:
preprocessing_pipeline = ColumnTransformer(transformers = [
    ('continuous', NoTransformer(), continuous_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [ ]:
features = continuous_features + categorical_features

## Entrenamiendo por grilla

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [ ]:
pipeline = Pipeline(steps = [
    ('preprocessing', preprocessing_pipeline),
    ('estimator', LinearRegression())
])

In [ ]:
accepted_targets = [column for column in dataset.columns if 'Forward_' in column]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
params_grid =[
    {'estimator':[LinearRegression()]},
    {
        'estimator': [RandomForestRegressor()],
        'estimator__max_depth': range(5, 16, 5),
        'estimator__min_samples_split': [2, 5, 10, 15, 20]
    },
    {
        'estimator': [LGBMRegressor(random_state = 42, silent = True)],
        'estimator__n_estimators': range(10, 101, 10)
    },
    {
        'estimator': [CatBoostRegressor(verbose = False)],
    },
]

Como queremos estimar varios targets en los cuales hay nulos en distintos rangos de tiempo, haremos cross-validation en cada caso.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, train_test_split

In [ ]:
target = 'Forward_Return_1m'

model_data = dataset[features + [target]].dropna()
X = model_data[features]
y = model_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = False, train_size = 0.8)
cv = TimeSeriesSplit(n_splits = 2)
model = GridSearchCV(pipeline, params_grid, cv = cv,
                    scoring='neg_mean_squared_error',
                    return_train_score = True)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

mse = mean_squared_error(y_test, model.predict(X_test))
r2 = r2_score(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

mse, rmse

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

metrics = {
    'mse': [],
    'rmse': [],
    'ticker': []
}

for ticker in X_test.Ticker.unique():
    # Compute metrics by ticker
    mask = X_test.Ticker == ticker
    mse = mean_squared_error(y_test[mask], model.predict(X_test[mask]))
    rmse = np.sqrt(mse)
    # Save in dict
    metrics['ticker'].append(ticker)
    metrics['mse'].append(mse)
    metrics['rmse'].append(rmse)

pd.DataFrame(metrics).sort_by("").head(10)


In [ ]:
import pickle
file = open('models/trained_model_1m.pkl', 'wb')
pickle.dump(model, file)
file.close()

In [ ]:
import pickle
file = open('data/test_data.pkl', 'wb')
pickle.dump(X_train, file)
file.close()

## Modelo por representation learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn  import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping

def create_dnn(units_a = 32, units_b = 16):
    n = len(features)
    model = Sequential()
    Dense(units_a, activation = 'relu', input_shape=(n,))
    model.add(Dense(units_b, activation = 'relu'))
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam")   
    return model